<a href="https://colab.research.google.com/github/dmachlanski/ncrm-causality-2021/blob/main/labs/day_2/Exercise_JOBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use the JOBS dataset. Here we evaluate against ATT and policy risk (R_pol).

In [ ]:
def calc_stats_effect(self, eff_pred):
    att, policy = self.evaluate_bin_att_effect(eff_pred)
    return att, policy

def evaluate_bin_att_effect(self, eff_pred):
    att = np.mean(self.yf[self.t > 0]) - np.mean(self.yf[(1 - self.t + self.e) > 1])

    att_pred = np.mean(eff_pred[(self.t + self.e) > 1])
    bias_att = att_pred - att

    policy_value = self.policy_val(eff_pred[self.e > 0])

    return np.abs(bias_att), 1 - policy_value
	
def policy_val(self, eff_pred):
    # Consider only the cases for which we have experimental data (i.e., e > 0)
    t = self.t[self.e > 0]
    yf = self.yf[self.e > 0]

    if np.any(np.isnan(eff_pred)):
        return np.nan, np.nan

    policy = eff_pred > 0.0
    treat_overlap = (policy == t) * (t > 0)
    control_overlap = (policy == t) * (t < 1)

    if np.sum(treat_overlap) == 0:
        treat_value = 0
    else:
        treat_value = np.mean(yf[treat_overlap])

    if np.sum(control_overlap) == 0:
        control_value = 0
    else:
        control_value = np.mean(yf[control_overlap])

    pit = np.mean(policy)
    policy_value = pit * treat_value + (1 - pit) * control_value

    return policy_value